# 1. 环境配置

## 1.1 python 环境准备

In [ ]:
! pip install openai==2.11.0 dashscope==1.25.4 langchain-classic==1.0.0 langchain==1.1.3 langchain-community==0.4.1 langchain-openai==1.1.3

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## 1.2 大模型密钥准备

请根据第一章内容获取相关平台的 API KEY，如若未在系统变量中填入，请将 API_KEY 信息写入以下代码（若已设置请忽略）：

In [1]:
import os

# os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxx"
# os.environ["DASHSCOPE_API_KEY"] = "sk-yyyyyyyy"

# 2. LangChain 链详解

在 LangChain 中，**链（Chain）** 是指将多个组件（如提示词、模型、解析器等）按照一定的逻辑顺序组合在一起，从而实现端到端的任务处理。下面我们依次介绍主要的链类型和它们的演进。

## 2.1 LLMChain （早期版本，已被淘汰）

**LLMChain** 是 LangChain 早期的核心链，主要由四个部分组成：

1. **PromptTemplate**：提示词模板，定义输入结构
2. **LLM**：语言模型，执行推理
3. **OutputParser**：输出解析器，格式化模型结果
4. **memory**：聊天记忆，存储对话的过程

后面两部分我们将在后续的课程中进行讲解，那在下面的代码中，我们通过将 LLMChain 绑定 llm 和 ChatPromptTemplate 两部分内容，最终实现链的调用。

In [1]:
from langchain_openai import ChatOpenAI
from langchain_classic.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate
import os

# 1. 定义提示词模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个智能 AI 助手"),
    ("human", "当前问题：{current_question}")
])

# 2. 定义语言模型
llm = ChatOpenAI(
    model="ernie-3.5-8k",
    openai_api_key=os.environ.get("OPENAI_API_KEY"),
    base_url="https://aistudio.baidu.com/llm/lmapi/v3"
)

# 3. 构建 LLMChain
conversation = LLMChain(
    llm=llm,
    prompt=prompt
)

# 简单演示
question = "一句话人工智能的应用有哪些？"
response = conversation.invoke({"current_question": question})
print("模型回答:", response["text"])

C:\Users\76391\AppData\Local\Temp\ipykernel_26500\514214940.py:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  conversation = LLMChain(


模型回答: 人工智能应用超广泛啦，像智能语音助手、图像识别技术、自动驾驶汽车、智能医疗诊断、个性化推荐系统这些可都是它大显身手的地方呢。


但是自 LangChain v0.2 以后，LLMChain 不再被官方所推荐使用（并将在 v1.0 时被丢弃），转而是推出 LangChain 的一种表达式语言 LCEL。
LLMChain的主要问题在于：
- 记忆/工具/解析耦合度高：迁移别的模型或改输出格式容易牵一发而动全身。
- 扩展性差：一旦需要新增逻辑，开发者只能额外编写 Python 代码，无法在链中直接完成扩展。

## 2.2 LCEL（LangChain Expression Language）

### 2.2.1 LCEL 简介

**LCEL**（LangChain Expression Language）是 LangChain 在 0.2 版本中引入的一种全新构建链的方式。它使用管道符 `|` 将不同的组件灵活串联起来，让链式调用的可读性和扩展性大幅提升，真正实现了模块化、声明式的工作流设计。

### 2.2.2 RunnableLambda 封装函数节点

先简单解释一下什么是 RunnableLambda ：
- **Runnable**：遵循 LangChain 的统一接口标准，可以接入 LCEL 管道。
- **Lambda**：允许将任意 Python 函数（同步或异步）快速包装为一个可执行节点。


它本质上就是一个 **函数 → Runnable 的适配器**，让你只需传入一个简单函数，就能快速把它接到整个 LCEL 链路中。

#### 应用场景1：类型审查

我们可以设置一个简单的 lambda 函数 `lambda x: print("🧪 当前数据类型：", type(x)) or x` ，从而实现传入一个内容 `x` ，输出打印输入 x 的类型，然后把原样的 x 再传下去 `print("🧪 当前数据类型：", type(x)) or x` 。这样我们把这部分内容接入到合适的位置里，就能看到 Runnable 对象的输入和输出格式了。

这里我再多讲一些，就是为什么必须 `or x` ：

* 如果只有 `print(...)`，因为返回值是 `None`，后续节点会接收到 `None`，就不能正常输出我们想要的内容了。也就意味着管道运算符的前后格式不统一，无法将真实的信息沿着管道运算符运输了。
* 加上 `or x`，确保表达式的最终返回值是 `x` 本身。其实这样的写法等同于以下的函数写法：


In [2]:
def debug(x):
    print("🧪 当前数据类型：", type(x))
    return x

比如这里我们就可以看到 `prompt` 需要传入的格式是 `<class 'dict'>` 字典。

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
import os
prompt = ChatPromptTemplate.from_template("用一句话介绍 {topic}")
llm = ChatOpenAI(model="ernie-3.5-8k",
openai_api_key=os.environ.get("OPENAI_API_KEY"),
base_url="https://aistudio.baidu.com/llm/lmapi/v3")
debug = RunnableLambda(lambda x: print("🧪 当前数据类型：", type(x)) or x)
chain = debug | prompt | llm
print(chain.invoke({"topic":"广州"}).content)

🧪 当前数据类型： <class 'dict'>
广州是一座历史悠久、商贸繁荣、美食荟萃且兼具现代都市魅力与岭南文化特色的国际化大都市。


这里我们也就可以观察到 `prompt` 的输出和 `llm` 的输入应该是支持 `<class 'langchain_core.prompt_values.ChatPromptValue'> `。

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
import os
prompt = ChatPromptTemplate.from_template("用一句话介绍 {topic}")
llm = ChatOpenAI(model="ernie-3.5-8k",
openai_api_key=os.environ.get("OPENAI_API_KEY"),
base_url="https://aistudio.baidu.com/llm/lmapi/v3")
debug = RunnableLambda(lambda x: print("🧪 当前数据类型：", type(x)) or x)
chain = prompt | debug | llm
print(chain.invoke({"topic":"广州"}).content)

🧪 当前数据类型： <class 'langchain_core.prompt_values.ChatPromptValue'>
广州是一座历史悠久、商贸繁荣、美食荟萃且充满现代活力的国际化大都市。


这里我们也就可以观察到 `llm` 的输出应该是 `<class 'langchain_core.messages.ai.AIMessage'>` 。

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
import os
prompt = ChatPromptTemplate.from_template("用一句话介绍 {topic}")
llm = ChatOpenAI(model="ernie-3.5-8k",
openai_api_key=os.environ.get("OPENAI_API_KEY"),
base_url="https://aistudio.baidu.com/llm/lmapi/v3")
debug = RunnableLambda(lambda x: print("🧪 当前数据类型：", type(x)) or x)
chain = prompt | llm | debug
print(chain.invoke({"topic":"广州"}).content)

🧪 当前数据类型： <class 'langchain_core.messages.ai.AIMessage'>
广州是一座历史悠久、商贸繁荣、美食汇聚且兼具现代都市魅力与岭南文化特色的国际化大都市。


#### 应用场景2：插入函数

实际上 RunnableLambda 不仅仅能封装 Lambda 函数，正常的 python 函数也能够进行封装。比如下面我们就在输入的字典里加了一个表情符🔥。比如下面的例子里我们传入的是一个字典 `{"topic":"开心"}` ，那么当经过这个函数以后就会变成 `{"topic":"开心🔥"}` 。当然这个只是一个很简单的例子，没有太多实际用处，但是我们也还是可以根据需要的格式内容进行修改的。

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
import os
prompt = ChatPromptTemplate.from_template("用一句话介绍 {topic}")
llm = ChatOpenAI(model="ernie-3.5-8k",
openai_api_key=os.environ.get("OPENAI_API_KEY"),
base_url="https://aistudio.baidu.com/llm/lmapi/v3")

def add_fire(input):
    input["topic"] += "🔥"
    return input

chain = RunnableLambda(add_fire) | prompt | llm 

print(chain.invoke({"topic":"开心"}).content)

开心如燎原之火，瞬间点燃内心每一处角落，让生活满是灿烂光芒。


在 LangChain 中，除了使用 RunnableLambda 以外，我们还可以用一个装饰器 @chain 来定义一个函数成为 Runnable 对象。然后我们可以直接使用该函数名称载入到 LCEL 中。

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import chain
import os
prompt = ChatPromptTemplate.from_template("用一句话介绍 {topic}")
llm = ChatOpenAI(model="ernie-3.5-8k",
openai_api_key=os.environ.get("OPENAI_API_KEY"),
base_url="https://aistudio.baidu.com/llm/lmapi/v3")

@chain
def add_fire(input):
    input["topic"] += "🔥"
    return input

chain = add_fire | prompt | llm 

print(chain.invoke({"topic":"开心"}).content)

开心如燎原之火，瞬间点燃内心每一寸角落，让生活满是灿烂光芒与无尽欢畅。


### 2.2.3 使用 RunnableMap 并行处理多个字段

有些任务我们希望同时处理多个数据项，比如多个提示词或多个模型，这时候就可以使用 **RunnableMap** 来实现。 **RunnableMap** 是 LangChain LCEL 提供的一个特殊的 Runnable 节点，用于**并行执行多个子 Runnable**，并将它们的结果收集到一个字典中返回。它适合同时执行多个互不依赖的任务，最终把结果统一汇总。这个时候，一次 .invoke 就会有多个结果在输出上，并且会以对应的名称展示。

#### 工作原理

* 输入：一个数据 `x`
* 内部：将同一个输入 `x` 传递给多个子 Runnable，每个 Runnable 独立执行
* 输出：一个字典，键为子 Runnable 的名字，值为对应的执行结果

#### 代码示例

比如我们分别设置了两个提示词，然后让这两个提示词分别传给同一个大模型。运行下面的代码我们就会发现返回的结果就是一个字典，包含了 `a_result` 和 `b_result` 的两个键。那其对应回复的文本内容就在值里的 `content` 里进行保存：

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableMap
import os

prompt1 = ChatPromptTemplate.from_template("你是百度制作的大模型，一句话回答用户向你提出问题： {topic}")
prompt2 = ChatPromptTemplate.from_template("你是OpenAI制作的大模型，一句话回答用户向你提出问题： {topic}")
llm = ChatOpenAI(model="ernie-3.5-8k",
openai_api_key=os.environ.get("OPENAI_API_KEY"),
base_url="https://aistudio.baidu.com/llm/lmapi/v3")

chain = RunnableMap({
    "a_result": prompt1 | llm,
    "b_result": prompt2 | llm,
})

print(chain.invoke({"topic":"你是谁？"}))
print(chain.invoke({"topic":"你是谁？"})["a_result"].content)
print(chain.invoke({"topic":"你是谁？"})["b_result"].content)

{'a_result': AIMessage(content='我是百度公司研发的知识增强大语言模型，中文名是文心一言，英文名是ERNIE Bot。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 15, 'total_tokens': 35, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'ernie-3.5-8k', 'system_fingerprint': None, 'id': 'as-76cumv7epm', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b215d-dc79-7800-903e-12d3f999a92b-0', usage_metadata={'input_tokens': 15, 'output_tokens': 20, 'total_tokens': 35, 'input_token_details': {}, 'output_token_details': {}}), 'b_result': AIMessage(content='我是百度公司研发的文心一言，并非OpenAI制作的大模型哦。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 16, 'total_tokens': 30, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'ernie-3.5-8k', 'system_fingerprint': None, 'id': 'as-mc4t7x05r5', 

### 2.2.4 LCEL 总结

- 通过上面的介绍，我们可以看到，LCEL 的出现为 LangChain 带来了全新的工作流编排方式。它不仅让 Prompt → LLM → 解析 → 后处理 的链路表达更加直观，还通过 Runnable 接口、管道式语法 和 可组合组件，让我们可以随时接入记忆、检索、并行处理等复杂功能，极大提升了应用的灵活性和可扩展性。

- 相比早期的 LLMChain，LCEL 更像是一种 声明式工作流语言，它统一了输入输出规范，支持异步与流式执行，同时与 LangSmith 等工具深度集成，方便我们进行可视化调试与性能优化。未来随着 LangChain 生态的不断丰富，LCEL 也将成为构建智能体、多模型协同应用的核心基石。

## 2.3 组合链（SequentialChain）

在 LangChain 中，除了最基础的 **LLMChain（或现在推荐的 LCEL 链）** 之外，还可以把多个链组合在一起，形成一个更复杂的工作流，这就是**组合链（Sequential Chains）**。

### 2.3.1 组合链的概念

组合链允许**多个链顺序执行**，上一步的输出可以作为下一步的输入，直到形成一个完整的多步骤工作流。主要包括两类：

* 简单顺序链
* 普通顺序链

下面我们就来详细的介绍一下这两种链的使用方式：

### 2.3.2 SimpleSequentialChain

### 概念

* SimpleSequentialChain 是最基础的组合链。
* 所有子链共享同一个输入输出，按顺序依次执行。
* 上一个子链的输出会自动作为下一个子链的输入，无需手动指定变量。

#### 代码示例

在这个示例中，我们的目标是：通过输入一个主题，最终获取与该主题相关的关键词。但如果直接让大模型一步到位地生成关键词，往往不够精准。因此我们采用**分步骤拆解**的方式：

1. **文章撰写**：先让模型根据主题生成一段简短文章。
2. **摘要提取**：再让模型对文章进行摘要，保留核心信息。
3. **关键词提取**：最后从摘要中提取关键词，确保关键词的准确性和代表性。

这种“分而治之”的策略能让每个步骤聚焦于一个小任务，降低出错率，提升最终结果的质量。

接下来我们将三个子链用 LCEL 管道 `|` 串联起来，形成一个完整的多步骤工作流：

In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableLambda
import os

llm = ChatOpenAI(model="ernie-3.5-8k",
openai_api_key=os.environ.get("OPENAI_API_KEY"),
base_url="https://aistudio.baidu.com/llm/lmapi/v3")

prompt_1 = PromptTemplate.from_template("写一段关于{input}的简短文章。")
chain_1 = prompt_1 | llm

prompt_2 = PromptTemplate.from_template("请总结以下文章：{input}")
chain_2 = prompt_2 | llm

prompt_3 = PromptTemplate.from_template("请提取以下内容的关键词：{input}")
chain_3 = prompt_3 | llm

map_input = RunnableLambda(lambda x: {"input": x.content})

# 将三个子链串起来
overall_chain = (
    chain_1 |
    map_input | chain_2 |
    map_input | chain_3
)

# 调用整个链
result = overall_chain.invoke({"input": "人工智能教育"})
print(result.content)


关键词：
1. 数字化时代
2. 人工智能
3. AI教育
4. 复合型人才
5. 思维升级
6. 挑战
7. 生态体系


#### 技术细节

这里我补充几个子链串联的代码细节。首先之所以要在 `chain_1` 、`chain_2` 和 `chain_3` 之间添加上 `map_input` ，也就是 `RunnableLambda(lambda x: {"input": x.content})` 。其实主要原因是 `chain` 的输出格式是 `AIMessages` ，而 `chain` 的输入格式是字典。因此为了能够满足正常的输入和输出，我们就需要将每次 `chain` 的输出内容提取（也就是 `x.content` ）进行格式修改，并且将其改造成合适的字典格式，也就是 `{"input": x.content}` 。这样我们就能够子链都连接在一起并进行调用了。

另外，其实从本质上来说，直接用 lambda 函数也可以把这些内容串起来，比如 `chain_1 | (lambda x:{"input": x.content}) | chain_2 ` 这样，之所以要改成 `RunnableLambda` 的形式是因为其是 LangChain 标准化的 Runnable 节点，支持所有链式功能，可与 LCEL 管道无缝衔接，扩展性更强。并且后期还能通过回调和监听机制来获取信息。因此这里就还是使用 `RunnableLambda` 的形式。

### 2.3.3 Sequential Chain

#### 概念

**SequentialChain** 是 LangChain 提供的一种顺序组合链，它与 SimpleSequentialChain 的区别在于：SequentialChain 允许我们**显式地定义输入和输出变量**，并能在多个子链之间灵活传递数据。因此它更适合复杂场景，比如需要管理多个输入、多个输出，或者不同子链之间存在依赖关系的任务流。

#### 工作机制

* **输入变量**：由用户明确指定，可传递给不同子链。
* **子链执行**：每个子链的输出会自动保存到上下文，并可供后续子链调用。
* **输出变量**：最终返回时，可以选择需要暴露给外部的变量。

SequentialChain 的核心优势在于**可控性和灵活性**：你可以决定哪些变量作为输入，哪些作为输出，以及如何在链路中复用中间结果。

但是这种 Chain 其实和 LLMChain 一样都即将被 LCEL 所取代。在 LCEL 中，`SimpleSequentialChain` 和 `SequentialChain` 的差别被大大弱化，因为 LCEL 本身就通过管道符 `|` 支持**顺序组合**和**数据传递**。

* **简单顺序链**：在 LCEL 中天然存在，直接用 `a | b | c` 串起来即可，输入输出会自动顺序传递，相当于 SimpleSequentialChain 的功能。

* **普通顺序链**：在 LCEL 中可以用 `RunnablePassthrough.assign(...)`、`RunnableMap` 等机制来显式管理输入输出，功能上和 SequentialChain 类似，甚至更强大。




#### 代码示例

在这个示例中，我们的任务是：**基于用户评论，自动生成一条合适的商家回复**。这个任务看似简单，但如果直接让模型一步生成，往往会遇到以下问题：

* **准确性不足**：模型可能遗漏评论中的关键信息。
* **语言混淆**：如果评论不是中文，模型生成的回复可能不符合用户原本的语言习惯。
* **逻辑跳跃**：直接生成回复可能缺乏推理过程，导致风格不一致或答非所问。

为了解决上述问题，我们采用了**分步骤、多阶段**的处理方式：

1. **评论翻译**：无论评论原文是何种语言，先翻译成中文，方便后续处理。
2. **关键问题提取**：基于中文翻译，抽取评论里最核心的问题或诉求。
3. **语言识别**：检测原评论使用的语言，确保商家回复时用同样的语言输出。
4. **生成商家回复**：结合提炼的总结与语言信息，最终生成一条与用户语言一致的、逻辑清晰的商家答复。

这种分而治之的策略有几个好处：

* **保证准确性**：先翻译、再提炼，确保模型聚焦评论本身。
* **保持一致性**：通过语言识别，保证最终回复与用户的语言环境一致。
* **增强鲁棒性**：多步骤拆解使每个子任务都更简单，降低大模型“一步到位”时的偏差。

In [10]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import os
from langchain_core.runnables import RunnableMap, RunnableLambda

# 初始化模型
llm = ChatOpenAI(model="ernie-3.5-8k",
openai_api_key=os.environ.get("OPENAI_API_KEY"),
base_url="https://aistudio.baidu.com/llm/lmapi/v3")

# ===== Step 1: 翻译评论 =====
first_prompt = PromptTemplate.from_template("将以下评论翻译成中文：{Review}")
chain_one = first_prompt | llm  # LCEL 里用 | 代替 LLMChain

# ===== Step 2: 提取关键问题 =====
second_prompt = PromptTemplate.from_template("请提取评论中最关键的问题：{Chinese_Review}")
chain_two = second_prompt | llm

# ===== Step 3: 识别语言 =====
third_prompt = PromptTemplate.from_template("以下评论是用什么语言写的：{Review}")
chain_three = third_prompt | llm

# ===== Step 4: 生成后续回复 =====
fourth_prompt = PromptTemplate.from_template(
    "根据以下总结和指定语言，以商家的视角，写一条对应语言的后续回复（最终只需要输出一条）：\n\n总结: {summary}\n\n语言: {language}"
)
chain_four = fourth_prompt | llm

# ===== 组合逻辑：先串行，再传递必要变量 =====
overall_chain = (
    RunnableMap({
        "summary": chain_one | RunnableLambda(lambda x: {"Chinese_Review": x.content}) | chain_two,  # 提取问题
        "language": chain_three                      # 识别语言
    })
    | RunnableLambda(lambda x: {
        "summary": x["summary"].content,
        "language": x["language"].content
    })
    | chain_four                                    # 生成后续回复
)

# 运行
result = overall_chain.invoke({"Review": "我好喜欢这个产品呀！"})
print(result.content)

谢谢您的喜爱呀，能得到您这么高的评价我们超开心的！之后我们也会继续努力，给大家带来更多优质产品哒！


#### 思路总结与扩展

通过这种方式，我们实现了一个**多阶段处理管道**：

* **输入端**只需提供一条评论
* **中间步骤**分别负责翻译、问题提取、语言识别
* **输出端**生成一条合适的商家回复

未来我们还可以参考这个方式，实现更多分支、并行处理或更复杂的多步骤工作流，例如：

* 多语言场景下的不同文化适配
* 同时生成商家回复与内部工单
* 并行执行情感分析和用户画像提取

In [ ]:
from langchain_core.messages import trim_messages